In [101]:
from notebook.notebookapp import list_running_servers
import requests
import json

In [105]:
servers = list(list_running_servers())
for s in servers:
    print(s)
lk = 'http://localhost:8888'+servers[0]['base_url']+'api/sessions'
r = requests.get(lk)
# json.loads(r.text)

{'base_url': '/', 'hostname': 'localhost', 'notebook_dir': '/Users/eyam/jupyter/jupyterLabIDE/submit_jobs', 'password': False, 'pid': 57269, 'port': 8888, 'secure': False, 'token': '', 'url': 'http://localhost:8888/'}


In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%pyth

In [ ]:
%load_ext submit_jobs.hysds_magic

In [5]:
%ls submit_jobs/hysds_magic

hysds.py


In [191]:
from IPython.core.magic import (Magics, magics_class, line_magic, cell_magic)
import requests
import json

@magics_class
class HysdsMagic(Magics):

    @line_magic
    def execute(self, line):
        lk = 'http://localhost:8888'
        endpoint = '/hysds/execute'
        id = 'org.n52.wps.server.algorithm.SimpleBufferAlgorithm'
        algo_ver,params = line.split('(')
        params = params[:-1]
        algo,ver = algo_ver.split(':')
        inputs = []
        for kvpair in params.split(','):
            k,v = kvpair.split('=')
            inputs.append(k)
        inputs.append('')
        call = '?algo_id={algo}&version={ver}&identifier={id}&inputs={inputs}&{params}'.format(algo=algo,ver=ver,id=id,inputs=','.join(inputs),params=params.replace(',','&'))
        url = lk + endpoint + call
        r = requests.get(url)
        resp = json.loads(r.text)
        print('{}\n{}'.format(url,resp['result']))
        return
    
    @line_magic
    def status(self, line):
        lk = 'http://localhost:8888'
        endpoint = '/hysds/getStatus'
        call = '?job_id={}'.format(line.strip())
        url = lk + endpoint + call
        r = requests.get(url)
        resp = json.loads(r.text)
        print('{}\n{}'.format(url,resp['result']))
        return
    
    @line_magic
    def result(self, line):
        lk = 'http://localhost:8888'
        endpoint = '/hysds/getResult'
        call = '?job_id={}'.format(line.strip())
        url = lk + endpoint + call
        r = requests.get(url)
        resp = json.loads(r.text)
        print('{}\n{}'.format(url,resp['result'].replace('<br>','\n')))
        return
    
    @line_magic
    def test(self,line):
        print(line)
        return

In [192]:
ip = get_ipython()
m = HysdsMagic()
ip.register_magics(m)

In [193]:
# %execute algo:version(p1=v1,p2=v2)
%execute plot_algo:master(pass_number=3,username=liz)

http://localhost:8888/hysds/execute?algo_id=plot_algo&version=master&identifier=org.n52.wps.server.algorithm.SimpleBufferAlgorithm&inputs=pass_number,username,&pass_number=3&username=liz
JobID is 60fb73a7-5e16-4752-a3a6-9f5b7a84d037


In [189]:
%status 569de45f-80eb-45be-8567-f45c743f7517

http://localhost:8888/hysds/getStatus?job_id=569de45f-80eb-45be-8567-f45c743f7517
JobID is 569de45f-80eb-45be-8567-f45c743f7517
Status: job-completed


In [194]:
%result 569de45f-80eb-45be-8567-f45c743f7517

http://localhost:8888/hysds/getResult?job_id=569de45f-80eb-45be-8567-f45c743f7517
JobID is 569de45f-80eb-45be-8567-f45c743f7517
ProductName: output-2019-05-07_23-15-51
Locations: http://maapdev.s3.amazonaws.com/products/plot/v1.0/2019/05/07/output-2019-05-07_23-15-51
	s3://s3.amazonaws.com:80/maapdev/products/plot/v1.0/2019/05/07/output-2019-05-07_23-15-51
	<a href="https://s3.console.aws.amazon.com/s3/buckets/maapdev/products/plot/v1.0/2019/05/07/output-2019-05-07_23-15-51/?region=us-east-1&tab=overview">https://s3.console.aws.amazon.com/s3/buckets/maapdev/products/plot/v1.0/2019/05/07/output-2019-05-07_23-15-51/?region=us-east-1&tab=overview</a>




In [188]:
d = {'hi':'bi'}
k = d.keys()
%test $k
%test {d.keys()}

dict_keys(['hi'])
dict_keys(['hi'])
